1. Import great_expectations and instantiate your Data Context

In [13]:
import datetime

import pandas as pd

import great_expectations as ge
import great_expectations.jupyter_ux
from great_expectations.core.batch import BatchRequest
from great_expectations.checkpoint import SimpleCheckpoint
from great_expectations.exceptions import DataContextError
context = ge.DataContext()

2. Instantiate two Validators, one for each Data Asset

In [14]:
batch_request_1 = BatchRequest(
 datasource_name="my_datasource",
 data_connector_name="default_inferred_data_connector_name",
 data_asset_name="Book1.csv"
)
upstream_validator = context.get_validator(batch_request=batch_request_1, expectation_suite_name="file_one")

batch_request_2 = BatchRequest(
 datasource_name="my_datasource",
 data_connector_name="default_inferred_data_connector_name",
 data_asset_name="Book2.csv"
)
downstream_validator = context.get_validator(batch_request=batch_request_2, expectation_suite_name="file_two")

3.Disable interactive evaluation for the downstream Validator

In [15]:
downstream_validator.interactive_evaluation = False

4. Define an Expectation using an Evaluation Parameter on the downstream Validator

In [16]:
# eval_param_urn = 'urn:great_expectations:validations:first_file:expect_table_row_count_to_be_between.result.observed_value'
# downstream_validator.expect_table_row_count_to_equal(
#    value={
#       '$PARAMETER': eval_param_urn, # this is the actual parameter we're going to use in the validation
#    }
# )
eval_param_urn = 'urn:great_expectations:validations:first_file:expect_column_distinct_values_to_contain_set.result.observed_value'
# eval_param_urn = 'urn:great_expectations:stores:ge_table_metric_store:substream_value'
downstream_validator.expect_column_values_to_be_in_set(
   "AccountID",
   ["ABC", "RE233"]
      # {"$PARAMETER":eval_param_urn, # this is the actual parameter we're going to use in the validation
   # }]
)

{
  "success": null,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {}
}

In [17]:
# eval_param_urn = 'urn:great_expectations:stores:ge_table_metric_store:substream_value'
eval_param_urn = 'urn:great_expectations:validations:first_file:expect_column_distinct_values_to_contain_set.result.observed_value'
downstream_validator.expect_column_values_to_be_in_set(
   "AccountID",
   [{"$PARAMETER":eval_param_urn, # this is the actual parameter we're going to use in the validation
   }]
)

{
  "success": null,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {}
}

In [18]:
print(downstream_validator.columns)

<bound method Validator.columns of <great_expectations.validator.validator.Validator object at 0x000001ADC21CFE50>>


5. Save your Expectation Suite

In [19]:
downstream_validator.save_expectation_suite(discard_failed_expectations=False)

2022-10-14T12:04:56+0300 - INFO - 	1 expectation(s) included in expectation_suite.


6. Execute an existing Checkpoint

In [20]:
results = context.run_checkpoint(
  checkpoint_name="file_one_check"
)

2022-10-14T12:04:56+0300 - INFO - 	2 expectation(s) included in expectation_suite.


Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

c:\Users\Admin\anaconda3\lib\site-packages\jinja2\environment.py:1088: DeprecationWarning: 'soft_unicode' has been renamed to 'soft_str'. The old name will be removed in MarkupSafe 2.1.
  return concat(self.root_render_func(self.new_context(vars)))


7. Rebuild Data Docs and review results in docs

In [21]:
context.build_data_docs()

{'local_site': 'file://d:\\Code\\GE\\two_files\\great_expectations\\uncommitted/data_docs/local_site/index.html'}